### Musical Track Database Assignment

**Objective**:  
Create a normalized SQLite database from an iTunes export file in CSV format. The structure of the database should include the following tables: **Artist**, **Genre**, **Album**, and **Track**.

**Task**:  
You will write a program that processes the CSV file `tracks.csv` and stores the data in an SQLite3 database with the following structure:

#### Database Structure:

```sql
CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title TEXT UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
```

### Instructions:

#### 1. **Create the Database**:
Create the above tables in your SQLite database.

#### 2. **Process the CSV File**:
The file `tracks.csv` contains the following columns (based on an example CSV file):
- **Track name**
- **Artist name**
- **Album title**
- **Genre**
- **Track length**
- **Track rating**
- **Track play count**

#### 3. **Insert the Data into the Tables**:
Insert the data into **Artist**, **Genre**, **Album**, and **Track** tables.

- Ensure that **Artist** names are stored only once and referenced by **Album**.
- **Genre** names should be stored only once and referenced by **Track**.
- **Album** should reference **Artist** and **Track** should reference **Album** and **Genre**.

#### 4. **Ensure Data Integrity**:
- Use **foreign keys** to link **Album** and **Track** with **Artist** and **Genre**.
- Use **INSERT OR IGNORE** to avoid duplicates.

#### 5. **Upload the Database**:
After running the program and inserting data, upload the SQLite3 database (`.sqlite`) here.

---

### Grading Query:

To grade the assignment, the following query will be used to verify your data:

```sql
SELECT Track.title, Artist.name, Album.title, Genre.name 
    FROM Track JOIN Genre JOIN Album JOIN Artist 
    ON Track.genre_id = Genre.ID and Track.album_id = Album.id 
        AND Album.artist_id = Artist.id
    ORDER BY Artist.name LIMIT 3
```

### Grading Query:

To grade the assignment, the following query will be used to verify your data:

```sql
SELECT Track.title, Artist.name, Album.title, Genre.name 
    FROM Track JOIN Genre JOIN Album JOIN Artist 
    ON Track.genre_id = Genre.ID and Track.album_id = Album.id 
        AND Album.artist_id = Artist.id
    ORDER BY Artist.name LIMIT 3
```

## What the Grading Query Should Return:

The result of the query should be similar to the following, and this is what you will use to verify that your data has been inserted correctly:

| Track                                   | Artist | Album       | Genre |
|-----------------------------------------|--------|-------------|-------|
| For Those About To Rock (We Salute You) | AC/DC  | Who Made Who| Rock  |
| Hells Bells                             | AC/DC  | Who Made Who| Rock  |
| Shake Your Foundations                  | AC/DC  | Who Made Who| Rock  |


### Here is the answer:

In [4]:
import sqlite3

# Connect to SQLite database (creates if doesn't exist)
conn = sqlite3.connect('music_database.sqlite')
cur = conn.cursor()

# Step 1: Create Tables
cur.execute('''CREATE TABLE IF NOT EXISTS Artist (
    id INTEGER PRIMARY KEY AUTOINCREMENT, 
    name TEXT UNIQUE)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Genre (
    id INTEGER PRIMARY KEY AUTOINCREMENT, 
    name TEXT UNIQUE)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Album (
    id INTEGER PRIMARY KEY AUTOINCREMENT, 
    artist_id INTEGER, 
    title TEXT UNIQUE)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Track (
    id INTEGER PRIMARY KEY AUTOINCREMENT, 
    title TEXT UNIQUE, 
    album_id INTEGER, 
    genre_id INTEGER, 
    len INTEGER, 
    rating INTEGER, 
    count INTEGER)''')

# Commit the table creation
conn.commit()


In [5]:
# Open the CSV file for reading
handle = open("tracks.csv")

# Loop through each line in the file
for line in handle:
    line = line.strip()
    pieces = line.split(',')
    if len(pieces) < 5: continue

    name = pieces[0]
    artist = pieces[1]
    album = pieces[2]
    count = pieces[3]
    rating = pieces[4]
    length = pieces[5]

    # Insert artist into Artist table (if not already present)
    cur.execute('''INSERT OR IGNORE INTO Artist (name) VALUES (?)''', (artist,))

    # Get the artist_id
    cur.execute('''SELECT id FROM Artist WHERE name = ?''', (artist,))
    artist_id = cur.fetchone()[0]

    # Insert album into Album table (if not already present)
    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) VALUES (?, ?)''', (album, artist_id))

    # Get the album_id
    cur.execute('''SELECT id FROM Album WHERE title = ?''', (album,))
    album_id = cur.fetchone()[0]

    # Insert genre into Genre table (if not already present)
    cur.execute('''INSERT OR IGNORE INTO Genre (name) VALUES (?)''', (pieces[6],))  # Assuming genre is in the 7th column

    # Get the genre_id
    cur.execute('''SELECT id FROM Genre WHERE name = ?''', (pieces[6],))
    genre_id = cur.fetchone()[0]

    # Insert track into Track table
    cur.execute('''INSERT OR REPLACE INTO Track (title, album_id, genre_id, len, rating, count) VALUES (?, ?, ?, ?, ?, ?)''',
                (name, album_id, genre_id, length, rating, count))

# Commit the changes to the database
conn.commit()

# Close the connection
conn.close()
